# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [2]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [3]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

# load dataset
dataset = load_dataset(dataset_name)

# print dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [4]:
# TODO: Print a record
print(dataset.shape)

{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}


In [8]:
# Print a record
idx = 5000

for k, v in dataset['train'][idx].items():
   print(k)
   print(v)
   print()

id
train_5000

dialogue
#Person1#: do you like animals? I really like dogs.
#Person2#: so do i. I don't like cats.
#Person1#: why? I think cats are ok.
#Person2#: I can't bear being near cats. They don't seem to like me either.
#Person1#: I like wild animals. I don't like spiders and snakes. I think spiders and snakes are disgusting.
#Person2#: I'm fond of snakes. I think they're great. I agree with you about spiders though. I think spiders are horrible. I think it's because they have so many legs.
#Person1#: I think bears are wonderful. Pandas are fantastic. I low the people who kill them for their fur.
#Person2#: I agree. I'm carry about mice. I think they're so cute!
#Person1#: really? I don't see the attraction. I'm afraid of mice.

summary
#Person1# likes dogs, wild animals but doesn't like spiders and snakes. #Person2# doesn't like cats but likes snakes and mice.

topic
animals



## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [9]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [10]:
# TODO: Load model
orig_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)


In [11]:
# TODO: Print number of trainable parameters
print(print_trainable_model_params(orig_model))

Trainable parameters: 247577856
Total parameters: 247577856
Percentable of trainable parameters: 100.00%


### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [13]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [14]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_dataset = dataset.map(tokenize_fn, batched=True)



Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
# TODO: Verify prepared data
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})


In [17]:
idx = 100

for k, v in tokenized_dataset['test'][idx].items():
   print(k)
   print(v)
   print()

id
test_33_2

dialogue
#Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person2#: What was the problem that time?
#Person1#: The feeling was all wrong, Mike. She is telling you that she doesn't want to see you any more, but I want to get more anger from you. You're acting hurt and sad, but that's not how your character would act in this situation.
#Person2#: But Jason and Laura have been together for three years. Don't you think his reaction would be one of both anger and sadness?
#Person1#: At this point, no. I think he would react the way most guys would, and then later on, we would see his real feelings.
#Person2#: I'm not so sure about that.
#Person1#: Let's try it my way, and you can see how you feel when you're saying your lines. After that, if it still doesn't feel right, we can try something else.

summary
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.

topic
shooting

inpu

In [18]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_dataset = tokenized_dataset.remove_columns([ 'id', 'summary', 'dialogue', 'topic'])

In [20]:
# TODO: Verify dataset again
idx = 100

for k, v in tokenized_dataset['train'][idx].items():
   print(k)
   print(v)
   print()

input_ids
[12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 27, 43, 3, 9, 682, 28, 82, 4807, 5, 1713, 345, 13515, 357, 4663, 10, 363, 81, 34, 58, 1713, 345, 13515, 536, 4663, 10, 499, 4807, 65, 118, 91, 21, 8, 657, 471, 42, 78, 5, 1713, 345, 13515, 357, 4663, 10, 37, 4807, 19, 323, 269, 230, 5, 27, 183, 182, 8032, 5, 1713, 345, 13515, 536, 4663, 10, 366, 56, 34, 36, 464, 541, 58, 1713, 345, 13515, 357, 4663, 10, 94, 225, 36, 223, 30, 16, 8, 416, 1158, 13, 477, 5, 1713, 345, 13515, 536, 4663, 10, 531, 27, 341, 43, 12, 726, 21, 8, 4807, 58, 1713, 345, 13515, 357, 4663, 10, 101, 31, 60, 352, 12, 428, 25, 3, 9, 998, 298, 8, 4807, 19, 323, 5, 1713, 345, 13515, 536, 4663, 10, 264, 6, 27, 278, 31, 17, 43, 12, 726, 21, 34, 58, 1713, 345, 13515, 357, 4663, 10, 465, 6, 59, 552, 39, 4807, 639, 223, 30, 5, 1713, 345, 13515, 536, 4663, 10, 16036, 6, 2049, 21, 762, 5, 1713, 345, 13515, 357, 4663, 10, 148, 31, 60, 2222, 6, 11, 27, 22103, 21, 8, 25741, 5, 20698, 10, 1, 0, 0, 0, 0, 

### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [21]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

CUDA available:  False


## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [22]:
print(orig_model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [23]:
# TODO: Configure LoRA
r = 16
lora_config = LoraConfig(
   r=r,
   lora_alpha=r,
   target_modules=['q', 'v'],
   lora_dropout=0.05,
   task_type=TaskType.SEQ_2_SEQ_LM
)

In [24]:
# TODO: Add LoRA to the LLM model to be trained
lora_model = get_peft_model(orig_model, lora_config)

In [25]:
# TODO: Print number of parameters, compare LoRA to the original model
print(print_trainable_model_params(lora_model))

Trainable parameters: 1769472
Total parameters: 249347328
Percentable of trainable parameters: 0.71%


In [26]:
# TODO: Train model with LoRA
lora_train_args = TrainingArguments(
   output_dir="lora-train-summary",
   auto_find_batch_size=True,
   learning_rate=1e-3,
   num_train_epochs=1   
)

In [ ]:
# TODO: Create trainer and train model
lora_trainer = Trainer(
   model = lora_model,
   args = lora_train_args,
   train_dataset= tokenized_dataset['train'],
   eval_dataset= tokenized_dataset['validation']
)

# Start training 
lora_trainer.train()

### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [28]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

lora_base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

lora_model = PeftModel.from_pretrained(
   lora_base_model, # base model
   peft_dialogue_summary_checkpoint, # adaptation
   torch_dtype = torch.bfloat16,
   is_trainable=False 
)

print(print_trainable_model_params(lora_model))
print(lora_model)

Trainable parameters: 0
Total parameters: 251116800
Percentable of trainable parameters: 0.00%
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=Fa

## Evaluate LoRA model

In [29]:
# TODO: Evaluate LoRA model against the original 
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [30]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = dataset['test'][i]['dialogue']
   s = dataset['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

i = 0
i = 1
i = 2
i = 3
i = 4


,label,original_model_summary,lora_model_summary
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...


### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [32]:
# TODO: create ROUGE metrics
rouge = evaluate.load('rouge')

In [35]:
# TODO: Evaluate the original model's result
orig_result = rouge.compute(
   references=summaries, # the label
   predictions=orig_model_summaries,
   use_stemmer=True,
   use_aggregator=True
)

lora_result = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_stemmer=True,
   use_aggregator=True
)

In [37]:
print(orig_result)
print('--------------')
print(lora_result)

{'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.1301282051282051), 'rougeLsum': np.float64(0.1301282051282051)}
--------------
{'rouge1': np.float64(0.341935138032699), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.2693796526054591), 'rougeLsum': np.float64(0.27097500453912726)}


In [ ]:
# TODO: Evaluate with Bleu metrics
